# FLIP(01):  Advanced Data Science
**(Module 03: Natural Language Processing)**

---
- Materials in this module include resources collected from various open-source online repositories.
- You are free to use, but NOT allowed to change or distribute this package.

Prepared by and for 
**Student Members** |
2006-2018 [TULIP Lab](http://www.tulip.org.au)

---


# Session 00 - the Farm: Requests

The Requests library is vital to add to your data science toolkit. It’s a simple yet powerful HTTP library, which means you can use it to access web pages.

We call it The Farm because you’ll be using it to get the raw ingredients (i.e. raw HTML) for your dishes (i.e. usable data).

Its simplicity is definitely its greatest strength. It’s so easy use that you could jump right in without reading documentation.

For example, if you want to pull down the contents of a page, it’s as easy as:

In [ ]:
import requests
page = requests.get('http://examplesite.com')
contents = page.content

# Make a Request

Now, let’s try to get a webpage. For this example, let’s get GitHub’s public timeline:

In [ ]:
r = requests.get('https://api.github.com/events')

Now, we have a **Response** object called r. We can get all the information we need from this object.

Requests’ simple API means that all forms of HTTP request are as obvious. For example, this is how you make an HTTP POST request:

In [ ]:
r = requests.post("http://httpbin.org/post", data = {'key':'value'})

Nice, right? What about the other HTTP request types: PUT, DELETE, HEAD and OPTIONS? These are all just as simple:

In [ ]:
r = requests.put('http://httpbin.org/put', data = {'key':'value'})
r = requests.delete('http://httpbin.org/delete')
r = requests.head('http://httpbin.org/get')
r = requests.options('http://httpbin.org/get')

That’s all well and good, but it’s also only the start of what Requests can do.

# Passing Parameters In URLs

You often want to send some sort of data in the URL’s query string. If you were constructing the URL by hand, this data would be given as key/value pairs in the URL after a question mark, e.g. httpbin.org/get?key=val. Requests allows you to provide these arguments as a dictionary of strings, using the params keyword argument. As an example, if you wanted to pass key1=value1 and key2=value2 to httpbin.org/get, you would use the following code:

In [ ]:
payload = {'key1': 'value1', 'key2': 'value2'}
r = requests.get('http://httpbin.org/get', params=payload)

You can see that the URL has been correctly encoded by printing the URL:

In [ ]:
print(r.url)

Note that any dictionary key whose value is None will not be added to the URL’s query string.

You can also pass a list of items as a value:

In [ ]:
payload = {'key1': 'value1', 'key2': ['value2', 'value3']}
r = requests.get('http://httpbin.org/get', params=payload)

In [ ]:
print(r.url)

# Response Content

We can read the content of the server’s response. Consider the GitHub timeline again:

In [ ]:
import requests

In [ ]:
r = requests.get('https://api.github.com/events')
r.text

Requests will automatically decode content from the server. Most unicode charsets are seamlessly decoded.

When you make a request, Requests makes educated guesses about the encoding of the response based on the HTTP headers. The text encoding guessed by Requests is used when you access r.text. You can find out what encoding Requests is using, and change it, using the r.encoding property:

In [ ]:
r.encoding

In [ ]:
r.encoding = 'ISO-8859-1'

If you change the encoding, Requests will use the new value of r.encoding whenever you call r.text. You might want to do this in any situation where you can apply special logic to work out what the encoding of the content will be. For example, HTML and XML have the ability to specify their encoding in their body. In situations like this, you should use r.content to find the encoding, and then set r.encoding. This will let you use r.text with the correct encoding.

Requests will also use custom encodings in the event that you need them. If you have created your own encoding and registered it with the codecs module, you can simply use the codec name as the value of r.encoding and Requests will handle the decoding for you.

# Binary Response Content

You can also access the response body as bytes, for non-text requests:

In [ ]:
r.content

The gzip and deflate transfer-encodings are automatically decoded for you.

For example, to create an image from binary data returned by a request, you can use the following code:

In [ ]:
from PIL import Image
from io import BytesIO

In [ ]:
i = Image.open(BytesIO(r.content))

# JSON Response Content

There’s also a builtin JSON decoder, in case you’re dealing with JSON data:

In [ ]:
import requests

In [ ]:
r = requests.get('https://api.github.com/events')
r.json()

In case the JSON decoding fails, r.json() raises an exception. For example, if the response gets a 204 (No Content), or if the response contains invalid JSON, attempting r.json() raises ValueError: No JSON object could be decoded.

It should be noted that the success of the call to r.json() does not indicate the success of the response. Some servers may return a JSON object in a failed response (e.g. error details with HTTP 500). Such JSON will be decoded and returned. To check that a request is successful, use r.raise_for_status() or check r.status_code is what you expect.

# Raw Response Content

In the rare case that you’d like to get the raw socket response from the server, you can access r.raw. If you want to do this, make sure you set stream=True in your initial request. Once you do, you can do this:

In [ ]:
r = requests.get('https://api.github.com/events', stream=True)

In [ ]:
r.raw

In [ ]:
r.raw.read(10)

In general, however, you should use a pattern like this to save what is being streamed to a file:

In [ ]:
with open(spider_data, 'wb') as fd:
    for chunk in r.iter_content(chunk_size=128):
        fd.write(chunk)

Using Response.iter_content will handle a lot of what you would otherwise have to handle when using Response.raw directly. When streaming a download, the above is the preferred and recommended way to retrieve the content. Note that chunk_size can be freely adjusted to a number that may better fit your use cases.

<font color = 'red'>** Note **</font>

An important note about using Response.iter_content versus Response.raw. Response.iter_content will automatically decode the gzip and deflate transfer-encodings. Response.raw is a raw stream of bytes – it does not transform the response content. If you really need access to the bytes as they were returned, use Response.raw.

# Custom Headers

If you’d like to add HTTP headers to a request, simply pass in a dict to the headers parameter.

For example, we didn’t specify our user-agent in the previous example:

In [ ]:
url = 'https://api.github.com/some/endpoint'
headers = {'user-agent': 'my-app/0.0.1'}

In [ ]:
r = requests.get(url, headers=headers)

Note: Custom headers are given less precedence than more specific sources of information. For instance:

    * Authorization headers set with headers= will be overridden if credentials are specified in .netrc, which in turn will be overridden by the auth= parameter.
    * Authorization headers will be removed if you get redirected off-host.
    * Proxy-Authorization headers will be overridden by proxy credentials provided in the URL.
    * Content-Length headers will be overridden when we can determine the length of the content.

Furthermore, Requests does not change its behavior at all based on which custom headers are specified. The headers are simply passed on into the final request.

Note: All header values must be a string, bytestring, or unicode. While permitted, it’s advised to avoid passing unicode header values.

# More complicated POST requests

Typically, you want to send some form-encoded data — much like an HTML form. To do this, simply pass a dictionary to the data argument. Your dictionary of data will automatically be form-encoded when the request is made:

In [ ]:
payload = {'key1': 'value1', 'key2': 'value2'}

In [ ]:
r = requests.post("http://httpbin.org/post", data=payload)

In [ ]:
print(r.text)

You can also pass a list of tuples to the data argument. This is particularly useful when the form has multiple elements that use the same key:

In [ ]:
payload = (('key1', 'value1'), ('key1', 'value2'))
r = requests.post('http://httpbin.org/post', data=payload)
print(r.text)

There are times that you may want to send data that is not form-encoded. If you pass in a string instead of a dict, that data will be posted directly.

For example, the GitHub API v3 accepts JSON-Encoded POST/PATCH data:

In [ ]:
import json

url = 'https://api.github.com/some/endpoint'
payload = {'some': 'data'}

r = requests.post(url, data=json.dumps(payload))

Instead of encoding the dict yourself, you can also pass it directly using the json parameter (added in version 2.4.2) and it will be encoded automatically:

In [ ]:
>>> url = 'https://api.github.com/some/endpoint'
>>> payload = {'some': 'data'}

>>> r = requests.post(url, json=payload)

Note, the json parameter is ignored if either data or files is passed.

Using the json parameter in the request will change the Content-Type in the header to application/json.

# POST a Multipart-Encoded File

Requests makes it simple to upload Multipart-encoded files:

In [ ]:
url = 'http://httpbin.org/post'
files = {'file': open('report.xls', 'rb')}

r = requests.post(url, files=files)
r.text

You can set the filename, content_type and headers explicitly: 

In [ ]:
url = 'http://httpbin.org/post'
files = {'file': ('report.xls', open('report.xls', 'rb'), 'application/vnd.ms-excel', {'Expires': '0'})}

r = requests.post(url, files=files)
r.text

If you want, you can send strings to be received as files:

In [ ]:
url = 'http://httpbin.org/post'
files = {'file': ('report.csv', 'some,data,to,send\nanother,row,to,send\n')}

r = requests.post(url, files=files)
r.text

In the event you are posting a very large file as a multipart/form-data request, you may want to stream the request. By default, requests does not support this, but there is a separate package which does - requests-toolbelt. You should read the toolbelt’s documentation for more details about how to use it.

For sending multiple files in one request refer to the advanced section.

<font color = 'red'>** Warning **</font>

It is strongly recommended that you open files in binary mode. This is because Requests may attempt to provide the Content-Length header for you, and if it does this value will be set to the number of bytes in the file. Errors may occur if you open the file in text mode.


# Response Status Codes

We can check the response status code:

In [ ]:
r = requests.get('http://httpbin.org/get')
r.status_code

In [ ]:
r.status_code == requests.codes.ok

If we made a bad request (a 4XX client error or 5XX server error response), we can raise it with Response.raise_for_status():

In [ ]:
>>> bad_r = requests.get('http://httpbin.org/status/404')
>>> bad_r.status_code

In [ ]:
bad_r.raise_for_status()

In [ ]:
r.raise_for_status()

# Response Headers

We can view the server’s response headers using a Python dictionary:

In [ ]:
r.headers

The dictionary is special, though: it’s made just for HTTP headers. According to RFC 7230, HTTP Header names are case-insensitive.

So, we can access the headers using any capitalization we want:

In [ ]:
r.headers['Content-Type']

In [ ]:
r.headers.get('content-type')

It is also special in that the server could have sent the same header multiple times with different values, but requests combines them so they can be represented in the dictionary within a single mapping, as per RFC 7230:

    * A recipient MAY combine multiple header fields with the same field name into one “field-name: field-value” pair, without changing the semantics of the message, by appending each subsequent field value to the combined field value in order, separated by a comma.

# Cookies

If a response contains some Cookies, you can quickly access them:

In [ ]:
url = 'http://example.com/some/cookie/setting/url'
r = requests.get(url)

r.cookies['example_cookie_name']

To send your own cookies to the server, you can use the cookies parameter:

In [ ]:
url = 'http://httpbin.org/cookies'
cookies = dict(cookies_are='working')

r = requests.get(url, cookies=cookies)
r.text

Cookies are returned in a **RequestsCookieJar**, which acts like a dict but also offers a more complete interface, suitable for use over multiple domains or paths. Cookie jars can also be passed in to requests:

In [ ]:
>>> jar = requests.cookies.RequestsCookieJar()
>>> jar.set('tasty_cookie', 'yum', domain='httpbin.org', path='/cookies')
>>> jar.set('gross_cookie', 'blech', domain='httpbin.org', path='/elsewhere')
>>> url = 'http://httpbin.org/cookies'
>>> r = requests.get(url, cookies=jar)
>>> r.text

# Redirection and History

By default Requests will perform location redirection for all verbs except HEAD.

We can use the history property of the Response object to track redirection.

The Response.history list contains the Response objects that were created in order to complete the request. The list is sorted from the oldest to the most recent response.

For example, GitHub redirects all HTTP requests to HTTPS:

In [ ]:
>>> r = requests.get('http://github.com')

>>> r.url

In [ ]:
r.status_code

In [ ]:
r.history

If you’re using GET, OPTIONS, POST, PUT, PATCH or DELETE, you can disable redirection handling with the allow_redirects parameter:

In [ ]:
>>> r = requests.get('http://github.com', allow_redirects=False)

>>> r.status_code

In [ ]:
r.history

If you’re using HEAD, you can enable redirection as well:

In [ ]:
>>> r = requests.head('http://github.com', allow_redirects=True)

>>> r.url

>>> r.history

# Timeouts

You can tell Requests to stop waiting for a response after a given number of seconds with the timeout parameter. Nearly all production code should use this parameter in nearly all requests. Failure to do so can cause your program to hang indefinitely:

In [ ]:
requests.get('http://github.com', timeout=0.001)

** Note **

timeout is not a time limit on the entire response download; rather, an exception is raised if the server has not issued a response for timeout seconds (more precisely, if no bytes have been received on the underlying socket for timeout seconds). If no timeout is specified explicitly, requests do not time out.


# Errors and Exceptions

In the event of a network problem (e.g. DNS failure, refused connection, etc), Requests will raise a ConnectionError exception.

Response.raise_for_status() will raise an HTTPError if the HTTP request returned an unsuccessful status code.

If a request times out, a Timeout exception is raised.

If a request exceeds the configured number of maximum redirections, a TooManyRedirects exception is raised.

All exceptions that Requests explicitly raises inherit from requests.exceptions.RequestException.